In [1]:
import serial 
import time
import numpy as np
import io
import sys
import zwoasi as asi
import os
import datetime
from astropy.io import fits 
import ctypes as c
import imp

## Below are the functions for 
## A) setting the gain, exposure, brightness, and flip of the camera. Flip should always be set to 0
## B) Saving the image settings of the camera. This function is run within the experimental settings function and will save a text file containing this information
## C) Opening the devices, will give access to three components (FW, relay port, and camera)

## D) Establishing a current before starting the experiment. This requires user input to exit. On the HV relay, the power supply plugged into position B is considered "ON". 

## E) The experimental settings. The parameters for this function are
     1. The names given to the three devices
     2. How many times the FW should rotate between two positions
     3/4 The two positions to alternate between
     5. How long to wait in seconds in between a pair of pictures (ie 2 pictures are taken, now how long to wait until the next pair of pictures is captured)
     ** note on 5: there will always be an additional 8 seconds added to the delay time **

In [20]:


def camera_settings(a,b,e,d):
    global gain, exposure, brightness, flip
    gain = a
    exposure = b
    brightness = e
    flip = d
    return gain, exposure, brightness, flip

                            
def save_control_values(settings):
    date = datetime.datetime.now().strftime("%y%m%d") #this is a string
    filename = date + '_settings'
    filename += '.txt'
    with open(filename, 'w') as f:
        for k in sorted(settings.keys()):
            f.write('%s: %s\n' % (k, str(settings[k])))
    print('Camera settings saved to %s' % filename)

def open_devices():

   
    port = 'COM5' #port for FW
    port2 = 'COM4' #port for relay
    #open FW102 communication
    try:
        x = serial.Serial(port=port, baudrate = 9600, timeout=1)
    except serial.SerialException as i:
        print('Port {0} is not available: {1}'.format(port,i))
        x.close()
        return
    except OSError as i:
        print('{}'.format(i))
        x.close()
        return 
    
    FW = io.TextIOWrapper(io.BufferedRWPair(x,x))
    
    
    #get the starting position of the wheel
    FW.write('pos?\r')
    FW.flush()
    cp = FW.readlines()[1][:-1]
    print('the current positions upon opening is', cp)
    FW.flush()
    FW.isOpen = True
    
    #open relay 
    relayPort = serial.Serial(port2, 19200,timeout=1)
    
    #open camera
    num_cameras = asi.get_num_cameras()
    if num_cameras ==0:
        print("no cameras found")
        sys.exit(0)
    cameras_found = asi.list_cameras()
    if num_cameras ==1:
        camera_id = 0
        print('Found one camera: {}'.format(cameras_found[0]))
    #open camera object
    camera = asi.Camera(camera_id)
    #print camera info (ie properties, controls)
    camera_info = camera.get_camera_property()
    print('')
    print('Camera Controls')
    controls = camera.get_controls()
    for cn in sorted(controls.keys()):
        print('     {}'.format(cn))
        for k in sorted(controls[cn].keys()):
            print('    {0}:   {1}'.format(k, repr(controls[cn][k])))
   
    print('the FW102, relay, and camera are connected!')
    
    return FW, relayPort, camera

def current_establishment():
    print('turning high voltage relay to "ON", is it hooked up to the polarity you wish to start with?')
    input('hit enter if the correct polarity is set')
    print('')
    x = 'relay on 1 \n\r'
    y = x.encode('utf-8')
    relay.write(y)
    input('press enter when the desired current is achieved!')
    

def experiment_settings(FW, RY, cam, rotations, posA, posB, delay_seconds, switch_n):
    
    """ parameterns are as follows:
        FW - the FW102
        RY - variable name given to the relay port
        ** rotations will also be used to determine the number of pics **
        rotations = how many times the FW rotates between the two positions
        posA, posB - the positions the FW102 will alternate between
        delay_seconds - how long to wait between taken pairs of pictures 
        (at each wavelength)
    """
    #set camera settings 
    #gain, exposure, brightness, flip = camera_settings(1,100000,0,0)
    # Use minimum USB bandwidth permitted
    cam.set_control_value(asi.ASI_BANDWIDTHOVERLOAD, cam.get_controls()['BandWidth']['MinValue'])
    

    cam.disable_dark_subtract()
    
    cam.set_control_value(asi.ASI_GAIN, gain)
    cam.set_control_value(asi.ASI_EXPOSURE, exposure)
    cam.set_control_value(asi.ASI_BRIGHTNESS, brightness)
    cam.set_control_value(asi.ASI_FLIP, flip)
    
    #start the clock, get the date, set filenames
    start = time.time()
    
    P1 = '_577'
    P2 = '_655'
    #define the command words needed to communicate
    on = 'on'
    off = 'off'
    control = 'relay'
    read = 'read'
    space = ' '
    relay = '0'
    relay2 = '1'
    end = '\n\r'
    cycling_command = ['on', 'off']
    
    #these four variables should be equal in length
    x = [on]*rotations
    y = [off]*rotations
    w = ([posA]*rotations)
    z = ([posB]*rotations)
    polarity = cycling_command*rotations
    
    for i in range(len(x)):
        print('i =', i)
        date = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
        #turn the LED on
        command = control + space + x[i] +space + relay + space + end
        c = command.encode('utf-8')
        RY.write(c)
        #change the polarity every 15th set of pictures
        
        if i != 0 and i%switch_n ==0:
            print('SWITCHING POLARITY')
            print('status of HV relay is:', polarity[i])
            switch = control + space + polarity[i] +space + relay2 + space + end
            s = switch.encode('utf-8')
            RY.write(s)
        #make sure FW is in position A
        FW.write('pos={}\r'.format(w[i]))
        FW.flush()
        FW.write('pos?\r')
        FW.flush()
        npos = FW.readlines()[-2][:-1]
        print('position = ', npos)
        time.sleep(1) #FW is excrutiatingly slow
        #TAKE A PICTURE
        #first make sure the camera is not doing anything
        try:
            cam.stop_exposure()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            pass
        cam.set_image_type(asi.ASI_IMG_RAW16)
        cam.FIT_capture(filename = date +P1+'_e1_{}'.format(i))
       
        #TAKE SECOND PICTURE
        try:
            cam.stop_exposure()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            pass
        cam.set_image_type(asi.ASI_IMG_RAW16)
        cam.FIT_capture(filename = date +P1+'_e2_{}'.format(i))
        
        #flush the serial communication line
        FW.flush()
        #safety precaution to make sure the wheel has time to get command
        time.sleep(.5)
        
        #rotate the wheel 
        FW.write('pos={}\r'.format(z[i]))
        FW.flush()
        FW.write('pos?\r')
        FW.flush()
        npos = FW.readlines()[-2][:-1]
        print('position = ', npos)
        time.sleep(1)
        #TAKE A PICTURE
        try:
            cam.stop_exposure()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            pass
        cam.set_image_type(asi.ASI_IMG_RAW16)
        cam.FIT_capture(filename =date+P2+'_e1_{}'.format(i))
        
        #TAKE PICTURE NUMBER 2 
        try:
            cam.stop_exposure()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            pass
        cam.set_image_type(asi.ASI_IMG_RAW16)
        cam.FIT_capture(filename =date+P2+'_e2_{}'.format(i))
        
        #turn the LED off 
        time.sleep(0.5)
        command2 = control + space + y[i] +space + relay + space + end
        c = command2.encode('utf-8')
        RY.write(c)
        
        
        
        #wait designated time before repeating process
        time.sleep(delay_seconds)
    
    
    #get total experiment time info, save camera settings
    save_control_values(cam.get_control_values())
    endtime = (time.time()-start)/60
    FW.flush()
    FW.write('pos?\r')
    FW.flush()
    npos = FW.readlines()[-2][:-1]
    print('ending position = ', npos)
    print('Complete. Total time: {:.3f} minutes'.format(endtime))
    #make sure relay is off
    command3 = "relay"+" "+"off" + " "+"0"+ "" + "\n\r"
    c = command3.encode('utf-8')
    
    RY.write(c)
    time.sleep(0.2) 
    response2 = RY.read(25)
    print('the status of the relay is', response2.decode('utf-8'))

    #close the ports, requires special function for FW, ugh
    def end(x): #function to close the FW
        try:
            if (x.isOpen == True):
                x.close()
                #if x.isOpen:
                    #x.close()
                print('the port is now closed')
                
        except:
            if (x.isOpen==False):
                print('the port is already closed!')
                
        print('port closed!')
        return
    timeout = (cam.get_control_value(asi.ASI_EXPOSURE)[0] / 1000) * 2 + 500
    cam.default_timeout = timeout
    
    cam.close()
    end(FW)
    RY.close()
    imp.reload(asi)
    
    print("")
    print('all ports are closed. Experiment is concluded')



In [21]:
libfile = "C:\\Users\\Masserate\\Desktop\\ASI SDK\\lib\\x64\\ASICamera2.dll"
asi.init(libfile)

## Below are three cells that provide an example of the above functions being used for an experiments 

In [22]:
wheel, relay, camera = open_devices()

the current positions upon opening is 6
Found one camera: ZWO ASI120MM-S

Camera Controls
     AutoExpMaxExpMS
    ControlType:   11
    DefaultValue:   100
    Description:   'Auto exposure maximum exposure value(unit ms)'
    IsAutoSupported:   False
    IsWritable:   True
    MaxValue:   60000
    MinValue:   1
    Name:   'AutoExpMaxExpMS'
     AutoExpMaxGain
    ControlType:   10
    DefaultValue:   50
    Description:   'Auto exposure maximum gain value'
    IsAutoSupported:   False
    IsWritable:   True
    MaxValue:   100
    MinValue:   0
    Name:   'AutoExpMaxGain'
     AutoExpTargetBrightness
    ControlType:   12
    DefaultValue:   100
    Description:   'Auto exposure maximum brightness value'
    IsAutoSupported:   False
    IsWritable:   True
    MaxValue:   160
    MinValue:   50
    Name:   'AutoExpTargetBrightness'
     BandWidth
    ControlType:   6
    DefaultValue:   50
    Description:   'The total data transfer rate percentage'
    IsAutoSupported:   True
    

## set the camera settings

In [17]:
gain, exposure, brightness, flip = (1,600000,0,0)

## establish the initial current - requires two user inputs

In [18]:
current_establishment()

turning high voltage relay to "ON", is it hooked up to the polarity you wish to start with?
hit enter if the correct polarity is set

press enter when the desired current is achieved!


In [19]:
experiment_settings(wheel, relay, camera, 11, 1, 2, 5, 15)
# numbers represent # rotations, position 1, position 2, time delay between pictures
# rotate 2 times between positions 1 and 2, wait 10 seconds in between picture pairs
#every 15th set of pictures, switch the polarity

i = 0
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 1
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 2
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 3
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 4
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 5
SWITCHING POLARITY
status of HV relay is: off
position =  1
successfully saved .FITS file
successfully saved .FITS file
position =  2
successfully saved .FITS file
successfully saved .FITS file
i = 6
position =  1
successful

# <span style="color:purple" > If you wish to run additional experiments, you must move the pictures to a different folder and rerun cells starting from cell 7 </span> 